# Reinforcement Learning

# 1. Markov Decision Process

This notebook presents some examples of Markov Decison Processes, where an **agent** interacts with its **environment** and collects **rewards**. 

In all considered models, rewards are attached to reached **states**. In games for instance, the reward of a state is +1 if you win, -1 if you loose, 0 in all other cases.

In [ ]:
import numpy as np

In [ ]:
from model import Walk, Maze

Each model is an object of the class ``Environment`` storing the current state. 

In [ ]:
from model import Environment

In [ ]:
methods_environment = [method for method in dir(Environment) if '__' not in method]

In [ ]:
methods_environment

The agent interacts with the environment and collects rewards.

In [ ]:
from agent import Agent

In [ ]:
methods_agent = [method for method in dir(Agent) if '__' not in method]

In [ ]:
methods_agent

## Walk

We start with a walk in a square. Some states (to be found) have positive rewards. Some wind (unknown) disturbs the walk.

In [ ]:
# environment
model = Walk()

In [ ]:
model.display()

In [ ]:
model.Size

In [ ]:
# rewards (to be discovered)
model.Rewards

In [ ]:
# wind (move probabilities due to wind)
model.Wind

In [ ]:
state = model.state

In [ ]:
state

In [ ]:
model.get_actions(state)

In [ ]:
action = (0, 1)

In [ ]:
model.step(action)

In [ ]:
model.state

In [ ]:
model.display()

In [ ]:
# agent with random policy (default)
agent = Agent(model)

In [ ]:
state = model.state
action = agent.get_action(state)

In [ ]:
state

In [ ]:
action

In [ ]:
# interaction with the environment
reward, stop = model.step(action)

In [ ]:
reward

In [ ]:
stop

In [ ]:
# all possible actions
agent.get_actions(state)

In [ ]:
# policy of the agent
probs, actions = agent.policy(state)

In [ ]:
print(probs)

In [ ]:
print(actions)

In [ ]:
# an episode
stop, states, rewards = agent.get_episode(horizon=50)

In [ ]:
# the episode includes the initial state
len(states)

In [ ]:
# display
animation = model.display(states)

In [ ]:
animation

In [ ]:
# initial reward = 0 by convention
len(rewards)

In [ ]:
np.sum(rewards)

In [ ]:
# gains from initial state, with discount factor
gains = agent.get_gains(n_runs=10, horizon=100, gamma=0.9)

In [ ]:
np.mean(gains)

## To do

* Test the weighted random policy where the probability of each move is proportional to its weight.
* Is this policy better than the (pure) random policy?

In [ ]:
weights = {(0, 1): 2, (1, 0): 2, (0, -1): 1, (-1, 0): 1}

In [ ]:
def weighted_random_policy(state, weights=weights):
    """Weighted random policy. 
    Each action is selected in proportion to its weight. 
    If the weights of available actions are all zero, take uniform choice among these actions."""
    actions = Walk().get_actions(state)
    probs = np.array([weights[action] for action in actions])
    if np.sum(probs):
        probs = probs / np.sum(probs)
    else:
        probs = np.ones(len(actions)) / len(actions)
    return probs, actions

In [ ]:
agent = Agent(model, policy=weighted_random_policy)

In [ ]:
gains = agent.get_gains(n_runs=10, horizon=100, gamma=0.9)

In [ ]:
np.mean(gains)

## Maze

Now let's try to escape a maze!

In [ ]:
maze_map = np.load('maze.npy')

In [ ]:
init_state = (1, 0)
exit_state = (1, 20)
Maze.set_parameters(maze_map, init_state, [exit_state])

In [ ]:
model = Maze()

In [ ]:
model.display()

In [ ]:
model.state

In [ ]:
state = model.state
reward = model.get_reward(state)

In [ ]:
# -1 in any state except the exit
reward

In [ ]:
model.get_actions(state)

In [ ]:
action = (0, 1)

In [ ]:
model.step(action)

In [ ]:
model.display()

In [ ]:
# agent with random policy
agent = Agent(model)

In [ ]:
stop, states, rewards = agent.get_episode(horizon=200)

In [ ]:
animation = model.display(states)

In [ ]:
animation

In [ ]:
# time in the Maze
-np.sum(rewards)

## To do

* Test the weighted random policy.
* Are there weights for which you can escape the maze in less than 250 steps?

In [ ]:
model = Maze()

In [ ]:
# bias to the right
weights = {(0, 1): 5, (1, 0): 2, (-1, 0): 2, (0, -1): 1}

In [ ]:
def weighted_random_policy(state, weights=weights):
    """Weighted random policy. 
    Each action is selected in proportion to its weight. 
    If the weights of available actions are all zero, take uniform choice among these actions."""
    actions = Maze().get_actions(state)
    probs = np.array([weights[action] for action in actions])
    if np.sum(probs):
        probs = probs / np.sum(probs)
    else:
        probs = np.ones(len(actions)) / len(actions)
    return probs, actions

In [ ]:
agent = Agent(model)

In [ ]:
horizon = 250

In [ ]:
gains = agent.get_gains(n_runs=100, horizon=horizon)

In [ ]:
# times to escape
times = -np.array(gains) + 1

In [ ]:
np.mean(times < horizon)

In [ ]:
agent = Agent(model, policy=weighted_random_policy)

In [ ]:
stop, states, rewards = agent.get_episode(horizon=200)

In [ ]:
animation = model.display(states)

In [ ]:
animation

In [ ]:
gains = agent.get_gains(n_runs=100, horizon=horizon)

In [ ]:
# times to escape
times = -np.array(gains) + 1

In [ ]:
np.mean(times < horizon)

## Games

Finally, let's play games!<br>
Note that in most games:
* you play against an adversary (which is part of the environment),
* you may play first or second,
* when your adversary plays, you have only one possible action (let your adversary play),
* you can also impose an action to your adversary (useful for training).

We here consider [Tic-Tac-Toe](https://en.wikipedia.org/wiki/Tic-tac-toe), [Nim](https://en.wikipedia.org/wiki/Nim), [Connect Four](https://en.wikipedia.org/wiki/Connect_Four) and [Five in a row](https://en.wikipedia.org/wiki/Gomoku).
Feel free to add more :-)

In [ ]:
from model import TicTacToe, Nim, ConnectFour, FiveInRow

Each game is an object of the class ``Game``. 

In [ ]:
from model import Game

In [ ]:
methods_game = [method for method in dir(Game) if '__' not in method]

In [ ]:
methods_game

The method ``get_next_state`` allows you to get the next state for any (state, action) pair, without modifying the current state. This will be useful to learn good policies.

The method ``get_available_actions`` gives the available actions, ignoring the player.

In [ ]:
set(methods_game) - set(methods_environment)

## Tic-Tac-Toe

We start with Tic-Tac-Toe.

### Play first

In [ ]:
# game against a random player (default)
game = TicTacToe()

In [ ]:
game.display()

In [ ]:
# next player, board
game.state

In [ ]:
# your actions
game.get_actions(game.state)

In [ ]:
# next state given some move
game.get_next_state(game.state, (1, 1))

In [ ]:
# the state is not modified
game.state

In [ ]:
# you play at random (default)
agent = Agent(game)

In [ ]:
# you play as player 1 (default)
agent.player

In [ ]:
# your adversary plays as player -1
game.adversary.player

In [ ]:
state = game.state
action = agent.get_action(state)

In [ ]:
action

In [ ]:
reward, stop = game.step(action)

In [ ]:
# you're blue
game.display()

In [ ]:
game.state

In [ ]:
# all possible moves
game.get_available_actions(game.state)

In [ ]:
# your moves (not your turn)
agent.get_actions(game.state)

In [ ]:
action = agent.get_action(game.state)

In [ ]:
reward, stop = game.step(action)

In [ ]:
game.display()

In [ ]:
stop, states, rewards = agent.get_episode()

In [ ]:
animation = game.display(states)

In [ ]:
animation

In [ ]:
rewards

In [ ]:
gains = agent.get_gains()
np.unique(agent.get_gains(), return_counts=True)

### Play second

In [ ]:
# your adversary starts
game = TicTacToe(play_first=False)

In [ ]:
game.first_player

In [ ]:
# you still play at random
agent = Agent(game)

In [ ]:
stop, states, rewards = agent.get_episode()

In [ ]:
animation = game.display(states)

In [ ]:
# you're still blue, red starts
animation

In [ ]:
rewards

In [ ]:
gains = agent.get_gains()
np.unique(agent.get_gains(), return_counts=True)

### Control your adversary

You can force the actions of your adversary (useful for training).

In [ ]:
game = TicTacToe()

In [ ]:
actions = [(0, 0), (1, 1), (0, 2),  (2, 2), (0, 1)]

In [ ]:
for action in actions:
    game.step(action)

In [ ]:
game.show()

### One step ahead

In [ ]:
# your adversary is random
game = TicTacToe()

In [ ]:
# you play with one-step ahead policy
agent = Agent(game, policy='one_step')

In [ ]:
stop, states, rewards = agent.get_episode()

In [ ]:
animation = game.display(states)

In [ ]:
animation

In [ ]:
gains = agent.get_gains()
np.unique(agent.get_gains(), return_counts=True)

In [ ]:
# your adversary also looks one-step ahead
game = TicTacToe(adversary_policy='one_step')

In [ ]:
agent = Agent(game, policy='one_step')

In [ ]:
stop, states, rewards = agent.get_episode()

In [ ]:
animation = game.display(states)

In [ ]:
animation

In [ ]:
np.unique(agent.get_gains(), return_counts=True)

## Nim

### Random players

In [ ]:
# game against a random player (default)
game = Nim()

In [ ]:
game.state

In [ ]:
game.display()

In [ ]:
# player, board
game.state

In [ ]:
# you play at random
agent = Agent(game)

In [ ]:
state = game.state
action = agent.get_action(state)

In [ ]:
action

In [ ]:
reward, stop = game.step(action)

In [ ]:
game.display()

In [ ]:
stop, states, rewards = agent.get_episode()

In [ ]:
animation = game.display(states)

In [ ]:
animation

In [ ]:
rewards

In [ ]:
np.unique(agent.get_gains(), return_counts=True)

### One step ahead

In [ ]:
game = Nim(adversary_policy='one_step')

In [ ]:
agent = Agent(game, policy='one_step')

In [ ]:
np.unique(agent.get_gains(), return_counts=True)

## Connect Four

### Random players

In [ ]:
# game against a random player
game = ConnectFour()

In [ ]:
game.display()

In [ ]:
game.state

In [ ]:
# you play at random
agent = Agent(game)

In [ ]:
stop, states, rewards = agent.get_episode()

In [ ]:
animation = game.display(states)

In [ ]:
# you play yellow
animation

In [ ]:
np.unique(agent.get_gains(n_runs=10), return_counts=True)

### One step ahead

In [ ]:
game = ConnectFour(adversary_policy='one_step')

In [ ]:
agent = Agent(game, policy='one_step')

In [ ]:
stop, states, rewards = agent.get_episode()

In [ ]:
animation = game.display(states)

In [ ]:
animation

In [ ]:
np.unique(agent.get_gains(n_runs=10), return_counts=True)

## Five-in-a-row

### Random players

In [ ]:
game = FiveInRow()

In [ ]:
game.display()

In [ ]:
agent = Agent(game)

In [ ]:
stop, states, rewards = agent.get_episode()

In [ ]:
animation = game.display(states)

In [ ]:
animation

In [ ]:
np.unique(agent.get_gains(n_runs=5), return_counts=True)

### One step ahead

In [ ]:
game = FiveInRow()

In [ ]:
agent = Agent(game, policy='one_step')

In [ ]:
np.unique(agent.get_gains(n_runs=5), return_counts=True)

## Value function

The value function of a policy can be computed from Bellman's equation, provided the state space is not too large.

You can check this condition by listing all states.

In [ ]:
model = Walk()

In [ ]:
len(model.get_all_states())

## To do

* Evaluate the random policy in the ``Walk`` model and display the value function.
* Observe the impact of the wind on the value function.
* Display the value function of the random policy in the maze.

In [ ]:
# we use sparse transition matrices
from scipy import sparse

In [ ]:
class PolicyEvaluation:
    """Evaluation of a policy by dynamic programming.
    
    Parameters
    ----------
    model: object of class Environment
        Model.
    policy: function
        Policy of the agent.
    player: int
        Player for games (1 or -1, default = default player of the game).
    gamma: float
        Discount factor (between 0 and 1).
    n_eval: int
        Number of iterations of Bellman's equation for policy evaluation.
    """
    def __init__(self, model, policy='random', player=None, gamma=1, n_eval=100):
        self.model = model
        agent = Agent(model, policy, player)
        self.policy = agent.policy
        self.player = agent.player
        self.gamma = gamma
        self.n_eval = n_eval
        self.index_states()
        if self.n_states == 0:
            raise ValueError("Not applicable. The state space is too large.")
        self.get_rewards()
        self.get_transitions()
        
    def index_states(self):
        """Index all states."""
        self.states = self.model.get_all_states()
        self.n_states = len(self.states)
        self.state_id = {self.model.encode(state): i for i, state in enumerate(self.states)}
        
    def get_state_id(self, state):
        return self.state_id[self.model.encode(state)]

    def get_rewards(self):
        """Get the reward of each state."""
        rewards = np.zeros(self.n_states)
        for i, state in enumerate(self.states):    
            rewards[i] = self.model.get_reward(state)
        self.rewards = rewards
        
    def get_actions(self, state, player=None):
        if self.model.is_game():
            if player is None:
                player = self.player
            actions = self.model.get_actions(state, player)
        else:
            actions = self.model.get_actions(state)
        return actions
    
    def get_transitions(self):
        """Get the transitions (probabilities to move from one state to another) for each action."""
        actions = self.model.get_all_actions()
        transitions = {action: sparse.lil_matrix((self.n_states, self.n_states)) for action in actions}
        for i, state in enumerate(self.states):    
            actions = self.get_actions(state)
            for action in actions:
                probs, next_states = self.model.get_transition(state, action)
                indices = np.array([self.get_state_id(next_state) for next_state in next_states])
                transitions[action][i, indices] = np.array(probs)
        self.transitions = {action: sparse.csr_matrix(transition) for action, transition in transitions.items()}
            
    def evaluate_policy(self):
        """Evaluate a policy by iteration of Bellman's equation."""
        transitions = self.transitions
        # probability of each action over the states
        probs = {action: np.zeros(self.n_states) for action in transitions}
        for state in self.states:    
            i = self.get_state_id(state)
            for prob, action in zip(*self.policy(state)):
                probs[action][i] = prob
        # Bellman's equation
        values = np.zeros(self.n_states)
        for t in range(self.n_eval):
            next_values = self.rewards + self.gamma * values
            values = np.zeros(self.n_states)
            for action, transition in transitions.items():
                values += probs[action] * transition.dot(next_values)
        self.values = values
   

In [ ]:
model = Walk()
agent = Agent(model)
random_policy = agent.policy

In [ ]:
model.Wind

In [ ]:
algo = PolicyEvaluation(model, random_policy, gamma=0.9)

In [ ]:
algo.evaluate_policy()

In [ ]:
values = algo.values

In [ ]:
model.display_values(values)

In [ ]:
wind = {(0, 1): 0.75}
Walk.set_parameters(Walk.Size, Walk.Rewards, wind)

In [ ]:
model = Walk()

In [ ]:
algo = PolicyEvaluation(model, random_policy, gamma=0.9)

In [ ]:
algo.evaluate_policy()

In [ ]:
values = algo.values

In [ ]:
model.display_values(values)

In [ ]:
algo = PolicyEvaluation(model, random_policy, gamma=0.9)

In [ ]:
model = Maze()

In [ ]:
algo = PolicyEvaluation(model)

In [ ]:
algo.evaluate_policy()

In [ ]:
values = algo.values

In [ ]:
model.display_values(values)

## To do

* Evaluate the random policy and the one-step policy in some games, when possible.
* Do the same when the adversary plays with the one-step policy.
* Predict some first good moves for each of these game.

## Tic-Tac-Toe

In [ ]:
# random players
Game = TicTacToe
game = Game()

In [ ]:
algo = PolicyEvaluation(game)

In [ ]:
algo.evaluate_policy()

In [ ]:
values = algo.values

In [ ]:
game.display()

In [ ]:
i = algo.get_state_id(game.state)
values[i]

In [ ]:
game.step((0,0))

In [ ]:
game.display()

In [ ]:
i = algo.get_state_id(game.state)
values[i]

In [ ]:
# your adversary move
game.step((1,1))

In [ ]:
game.display()

In [ ]:
i = algo.get_state_id(game.state)
values[i]

In [ ]:
# best move in the initial state
state = game.init_state()

In [ ]:
action_value = dict()
for action in game.get_actions(state):
    next_state = game.get_next_state(state, action)
    i = algo.get_state_id(next_state)
    action_value[action] = values[i]

In [ ]:
action_value

In [ ]:
best_action = max(action_value, key=action_value.get)

In [ ]:
best_action

In [ ]:
game = Game()

In [ ]:
game.step(best_action)

In [ ]:
game.display()

In [ ]:
# one-step policy
game = Game()
algo = PolicyEvaluation(game, policy='one_step')
algo.evaluate_policy()

In [ ]:
values = algo.values
i = algo.get_state_id(game.state)
values[i]

In [ ]:
state = game.state

In [ ]:
action_value = dict()
for action in game.get_actions(state):
    next_state = game.get_next_state(state, action)
    i = algo.get_state_id(next_state)
    action_value[action] = values[i]

In [ ]:
action_value

In [ ]:
best_action = max(action_value, key=action_value.get)

In [ ]:
best_action

In [ ]:
# one-step players
game = Game(adversary_policy='one_step')
algo = PolicyEvaluation(game, policy='one_step')
algo.evaluate_policy()

In [ ]:
values = algo.values
i = algo.get_state_id(game.state)
values[i]

In [ ]:
state = game.state

In [ ]:
action_value = dict()
for action in game.get_actions(state):
    next_state = game.get_next_state(state, action)
    i = algo.get_state_id(next_state)
    action_value[action] = values[i]

In [ ]:
action_value

In [ ]:
best_action = max(action_value, key=action_value.get)

In [ ]:
best_action

## Nim

In [ ]:
Game = Nim

In [ ]:
# one-step players
game = Game(adversary_policy='one_step')
algo = PolicyEvaluation(game, policy='one_step')
algo.evaluate_policy()

In [ ]:
values = algo.values
i = algo.get_state_id(game.state)
values[i]

In [ ]:
state = game.state

In [ ]:
action_value = dict()
for action in game.get_actions(state):
    next_state = game.get_next_state(state, action)
    i = algo.get_state_id(next_state)
    action_value[action] = values[i]

In [ ]:
# all actions have the same value
action_value

In [ ]:
# another state, later in the match
game.state = (1, np.array([1, 3, 0, 0]))

In [ ]:
game.display()

In [ ]:
state = game.state

In [ ]:
action_value = dict()
for action in game.get_actions(state):
    next_state = game.get_next_state(state, action)
    i = algo.get_state_id(next_state)
    action_value[action] = values[i]

In [ ]:
# all actions have the same value
action_value

In [ ]:
best_action = max(action_value, key=action_value.get)

In [ ]:
best_action

In [ ]:
game.step(best_action)

In [ ]:
game.display()